In [1]:
# import os
import json
import itertools
from SPARQLWrapper import SPARQLWrapper, JSON
import numpy as np
import json
import pandas as pd
import threading


# Get the list of companies for which we need data
def get_dbpedia_sparql_data(query):
        sparql = SPARQLWrapper("http://dbpedia.org/sparql")
        sparql.setReturnFormat(JSON)
        sparql.setQuery(query)  # the previous query as a literal string
        return sparql.query().convert()

def returnDF(data):
    dataArray=[]
    colNames=data['head']['vars']
    for row in data['results']['bindings']:
        rowArray=[]
        for colName in colNames:
            rowArray.append(row[colName]['value'])
        dataArray.append(rowArray)
    dataArray=pd.DataFrame(dataArray,columns=colNames)
    return(dataArray)

In [4]:
# We will now get all the data for each of the company irrespective of location
firstCharacterString='ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789!@_#$^&='
companyListAllLocations=pd.DataFrame(columns=['CompanyName'])
for curCharacter in firstCharacterString:
    curCompanyListQueryAllLocations='''
    SELECT distinct ?a
    WHERE { ?a rdf:type <http://dbpedia.org/ontology/Company> .
    ?a rdfs:label ?aLabel
    filter(regex(REPLACE(?aLabel, '"', '', "i"),"^%s","i"))
    }
    ''' % format(curCharacter)
    curCompanyListAllLocations=returnDF(get_dbpedia_sparql_data(curCompanyListQueryAllLocations))
    curCompanyListAllLocations.columns=['CompanyName']
    companyListAllLocations=pd.concat([companyListAllLocations,curCompanyListAllLocations])
    print(companyListAllLocations.shape[0])
    
print("The total company List is {}".format(companyListAllLocations.shape[0]))

9130
16504
26454
31230
35562
40055
44825
49077
52834
55016
58030
62353
69826
73904
76384
82887
83386
87991
97991
105294
107028
109666
113017
113318
113956
114805
114810
114964
115117
115221
115281
115323
115343
115367
115393
115416
115416
115423
115423
115423
115423
125423
125424
125424
The total company List is 125424


In [ ]:
# We will now get all the data for each of the company irrespective of location
fileCounter=0
FullCompanyData=pd.DataFrame(columns=['property','hasValue','CompanyName'])
for curCompanyName in companyListAllLocations['CompanyName'].values:
    companyQuery='''SELECT ?property ?hasValue
    WHERE {
      { <%s> ?property ?hasValue }
      UNION
      { ?hasValue ?property <%s> }
    }''' % (curCompanyName,curCompanyName)
    curCompanyData=returnDF(get_dbpedia_sparql_data(companyQuery))
    curCompanyData['Company']=curCompanyName
    curCompanyData.columns=['property','hasValue','CompanyName']
    FullCompanyData=pd.concat([FullCompanyData,curCompanyData])
    if(FullCompanyData.shape[0] > 100000):    
        FullCompanyData.to_csv('/home/ubuntu/anant/data/CompanyDataSansLocation_{}.txt'.format(fileCounter),index=False,encoding='utf-8')
        fileCounter=fileCounter + 1
        FullCompanyData=pd.DataFrame(columns=['property','hasValue','CompanyName'])
        print("Written to file {}".format(fileCounter-1))

FullCompanyData.to_csv('/home/ubuntu/anant/data/CompanyDataSansLocation_{}.txt'.format(fileCounter),index=False,encoding='utf-8')
fileCounter=fileCounter + 1
FullCompanyData=pd.DataFrame(columns=['property','hasValue','CompanyName'])
print("Written to file {}".format(fileCounter-1))

Written to file 0
Written to file 1
Written to file 2
Written to file 3
Written to file 4
Written to file 5
Written to file 6
Written to file 7
Written to file 8
Written to file 9
Written to file 10
Written to file 11
Written to file 12
Written to file 13
Written to file 14
Written to file 15
Written to file 16
Written to file 17
Written to file 18
Written to file 19
Written to file 20
Written to file 21
Written to file 22
Written to file 23
Written to file 24
Written to file 25
Written to file 26
Written to file 27
Written to file 28
Written to file 29
Written to file 30
Written to file 31
Written to file 32
Written to file 33
Written to file 34
Written to file 35
Written to file 36


In [ ]:
# Get the predicate data for all the RHS
predicateList=pd.read_csv('/home/ubuntu/anant/data/predicateListJoins.txt',header=-1)
predicateList.columns=['property']
predicateList['property']=predicateList['property'].map(lambda x : str(x).replace('<','').replace('>',''))

# Complete list of all predicates
predicateValues=[]

for curFileName in os.listdir('/home/ubuntu/anant/data/'):
    if('CompanyDataSansLocation' in curFileName ):
        FullCompanyData=pd.read_csv('/home/ubuntu/anant/data/{}'.format(curFileName))
        tempValues=list(FullCompanyData.merge(predicateList,left_on='property',right_on='property',how='inner')['hasValue'].unique())
        predicateValues=predicateValues + tempValues
        
predicateValues=list(set(predicateValues))

predicateData=pd.DataFrame(columns=['property','hasValue','Predicate'])
fileCounter=0

for curPredicate in predicateValues:
    predicateQuery='''SELECT ?property ?hasValue
    WHERE {
      { <%s> ?property ?hasValue }
      UNION
      { ?hasValue ?property <%s> }
    }''' % (curPredicate,curPredicate)
    try:
        curpredicateData=returnDF(get_dbpedia_sparql_data(predicateQuery))
        curpredicateData['Predicate']=curPredicate
        curpredicateData.columns=['property','hasValue','Predicate']
        predicateData=pd.concat([predicateData,curpredicateData])
    except:
        print("Error for {}".format(curPredicate))
    if(predicateData.shape[0] > 100000):    
        predicateData.to_csv('/home/ubuntu/anant/data/PredicateDataSansLocation_{}.txt'.format(fileCounter),index=False,encoding='utf-8')
        fileCounter=fileCounter + 1
        print("Written to file {} with shape {}".format(fileCounter-1,predicateData.shape[0]))
        predicateData=pd.DataFrame(columns=['property','hasValue','Predicate'])

predicateData.to_csv('/home/ubuntu/anant/data/PredicateDataSansLocation_{}.txt'.format(fileCounter),index=False,encoding='utf-8')
fileCounter=fileCounter + 1
print("Written to file {} with shape {}".format(fileCounter-1,predicateData.shape[0]))
predicateData=pd.DataFrame(columns=['property','hasValue','Predicate'])